In [7]:
import sys
import json
import glob
sys.path.append("..")

import pandas as pd
import numpy as np
import tqdm
import torch_geometric

from gvpgnn.datasets import ProteinGraphDataset, BatchSampler
from gvpgnn.paths import data_folder

In [2]:
dataset_version = "cleaned_skip_missing"
split_name = "train"

dataset = ProteinGraphDataset(
  data_folder(f"{dataset_version}/{split_name}"),
  num_positional_embeddings=16,
  top_k=30,
  num_rbf=16,
  device="cpu"
)

print("Dataset size:", len(dataset))

# Make sure that we can iterate through the dataset:
for i in range(min(len(dataset), 3)):
  print(dataset[i])

Dataset size: 3996
Data(x=[132, 3], edge_index=[2, 3960], seq=[132], name='4nleA02', task_label=0, node_s=[132, 326], node_v=[132, 3, 3], edge_s=[3960, 32], edge_v=[3960, 1, 3], mask=[132])
Data(x=[206, 3], edge_index=[2, 6180], seq=[206], name='1vkyB01', task_label=8, node_s=[206, 326], node_v=[206, 3, 3], edge_s=[6180, 32], edge_v=[6180, 1, 3], mask=[206])
Data(x=[96, 3], edge_index=[2, 2880], seq=[96], name='2gwnA01', task_label=2, node_s=[96, 326], node_v=[96, 3, 3], edge_s=[2880, 32], edge_v=[2880, 1, 3], mask=[96])


In [ ]:
dataset.class_weights
dataset.label_counts
sum(dataset.label_counts)
# len(dataset.node_counts)
dataset.class_weights

In [19]:
sampler = BatchSampler(
  dataset.node_counts,
  max_nodes=3000,
  sampler_weights=dataset.sampler_weights
)

dataloader = torch_geometric.data.DataLoader(
  dataset,
  num_workers=1,
  batch_sampler=sampler,
)

N = 100
label_freq = np.zeros(10)

for i, batch in enumerate(dataloader):
  print(i)
  if i > N:
    break
  batch_labels = batch.task_label.numpy()
  for label in batch_labels:
    label_freq[label] += 1

/Users/miloknowles/envs/cath-proteins-q1ibxFh1/lib/python3.11/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101


In [20]:
label_freq / label_freq.sum()

array([0.09928717, 0.09725051, 0.10285132, 0.10285132, 0.09368635,
       0.10743381, 0.09521385, 0.09775967, 0.10183299, 0.10183299])